# Individual Modeling Notebook

## Created by: Kathryn O'Connor

# Introduction
Introduction Home Credit is an international finance provider aiming to increase financial inclusion by providing financial loans to those with little to no credit history. Rather than rely on credit history, Home Credit uses a variety of data to assess an individual’s repayment abilities.

While they currently use a variety of statistical and machine learning methods to make their predictions, Home Credit is hoping to further harness their data to ensure that those who are capable of repaying loans are not rejected. Achieving greater accuracy on whether an individual is capable of loan repayment will not only allow Home Credit to broaden their customer base (and in turn, profit) they will also be able to help a larger population of individuals who are in need of loans.

In this notebook, I focused on upsampling of the logistic regression model aimed at tackling the inbalanced nature of the data set.

## Package Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE


import seaborn as sns
import matplotlib.pyplot as plt

## File Upload into Env

In [2]:
# Public URL after making the file public in the format 'https://storage.googleapis.com/...'
file_url = 'https://storage.googleapis.com/home_credit_files/application_train.csv'
# test_url = 'https://storage.googleapis.com/home_credit_files/application_test.csv'
# POS_CASH_balance_url = 'https://storage.googleapis.com/home_credit_files/POS_CASH_balance.csv'
# bureau_url = 'https://storage.googleapis.com/home_credit_files/bureau.csv'
# bureau_balance_url = 'https://storage.googleapis.com/home_credit_files/bureau.csv'
# credit_card_balance = 'https://storage.googleapis.com/home_credit_files/credit_card_balance.csv'
# installments_payments = 'https://storage.googleapis.com/home_credit_files/credit_card_balance.csv'
# previous_application = 'https://storage.googleapis.com/home_credit_files/previous_application.csv'


# Read the CSV directly from the URL
df = pd.read_csv(file_url)

#print(df.head())

/var/folders/4l/0tw9tkvj2gx_v0qrnty9rt0r0000gn/T/ipykernel_31513/3847801500.py:13: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_url)


## Exploratory Data Analysis

Below, I perform EDA on the train data set to prepare it for modeling. I began by analyzing the initial data set and obtaining the proportion of the target variable in the train set. About 92% of the data is 0, indicating those that repayed their loans on time. About 8% represent 1, those that did not repay on time.

In [3]:
print(df.shape)

(307511, 122)


In [4]:
df['TARGET'].value_counts()

TARGET
0    282686
1     24825
Name: count, dtype: int64

In [5]:
df['TARGET'].value_counts(normalize = True) * 100

TARGET
0    91.927118
1     8.072882
Name: proportion, dtype: float64

### Evaluating NAs/missing values

Next, we explored missing variables. We calculated the percentage of missing values per column, and after analyzing the descriptions for each, decided to remove those that were missing greater than 10% of their values. Many of these variables were likely to be filled with NAs because they were simply not applicable to the applicant (such as COMMONAREA_AVG which included 'normalized information about building where the client lives' or because they were not required questions for the application.

In [6]:
# Train data set

# calculate total number of missing values for each column
missing_values_train = df.isnull().sum()

# calculate total number of rows
total_rows_train = df.shape[0]

# calculate percentage of missing values for each column
pct_missing_train = (missing_values_train / total_rows_train) * 100

# sort output
pct_missing_sorted_train = pct_missing_train.sort_values(ascending=False)

# print result
print(pct_missing_sorted_train.head(25))

COMMONAREA_MEDI             69.872297
COMMONAREA_AVG              69.872297
COMMONAREA_MODE             69.872297
NONLIVINGAPARTMENTS_MODE    69.432963
NONLIVINGAPARTMENTS_AVG     69.432963
NONLIVINGAPARTMENTS_MEDI    69.432963
FONDKAPREMONT_MODE          68.386172
LIVINGAPARTMENTS_MODE       68.354953
LIVINGAPARTMENTS_AVG        68.354953
LIVINGAPARTMENTS_MEDI       68.354953
FLOORSMIN_AVG               67.848630
FLOORSMIN_MODE              67.848630
FLOORSMIN_MEDI              67.848630
YEARS_BUILD_MEDI            66.497784
YEARS_BUILD_MODE            66.497784
YEARS_BUILD_AVG             66.497784
OWN_CAR_AGE                 65.990810
LANDAREA_MEDI               59.376738
LANDAREA_MODE               59.376738
LANDAREA_AVG                59.376738
BASEMENTAREA_MEDI           58.515956
BASEMENTAREA_AVG            58.515956
BASEMENTAREA_MODE           58.515956
EXT_SOURCE_1                56.381073
NONLIVINGAREA_MODE          55.179164
dtype: float64


In [7]:
missings = pct_missing_sorted_train.to_frame(name='MissingPercentage')
print(missings)

                          MissingPercentage
COMMONAREA_MEDI                   69.872297
COMMONAREA_AVG                    69.872297
COMMONAREA_MODE                   69.872297
NONLIVINGAPARTMENTS_MODE          69.432963
NONLIVINGAPARTMENTS_AVG           69.432963
...                                     ...
NAME_HOUSING_TYPE                  0.000000
NAME_FAMILY_STATUS                 0.000000
NAME_EDUCATION_TYPE                0.000000
NAME_INCOME_TYPE                   0.000000
SK_ID_CURR                         0.000000

[122 rows x 1 columns]


### Removing columns with greater than 10% missing values

In [8]:
filter_missings = missings[(missings['MissingPercentage'] < 10)]
filter_missings.shape

(65, 1)

### Removing 'FLAG' columns and ID column

In [9]:
columns = list(filter_missings.index)
column_list = [item for item in columns if 'FLAG' not in item]
#column_list_2 = [item for item in column_list if 'TARGET' not in item]
column_list_3 = [item for item in column_list if 'SK_ID_CURR' not in item]

### Removing additional unhelpful columns

Further analyzing of column descriptions lead to the removal of additional columns, as shown below. We then created a fresh data frame with the columns we wanted to move forward with in our modeling.

In [10]:
list1 = column_list_3
list2 = ['NAME_TYPE_SUITE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']

# Initialize an empty list to store items from list1 that are not in list2
items_not_in_list2 = []

# Iterate through each item in list1
for item in list1:
    # Check if the item is not in list2
    if item not in list2:
        # Add the item to the items_not_in_list2 list
        items_not_in_list2.append(item)

# Print the items that are not in list2
items_not_in_list2

['EXT_SOURCE_2',
 'AMT_GOODS_PRICE',
 'AMT_ANNUITY',
 'CNT_FAM_MEMBERS',
 'DAYS_LAST_PHONE_CHANGE',
 'CNT_CHILDREN',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'AMT_CREDIT',
 'AMT_INCOME_TOTAL',
 'LIVE_CITY_NOT_WORK_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'TARGET',
 'REG_CITY_NOT_LIVE_CITY',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'REG_REGION_NOT_LIVE_REGION',
 'HOUR_APPR_PROCESS_START',
 'WEEKDAY_APPR_PROCESS_START',
 'REGION_RATING_CLIENT_W_CITY',
 'REGION_RATING_CLIENT',
 'ORGANIZATION_TYPE',
 'DAYS_ID_PUBLISH',
 'DAYS_REGISTRATION',
 'DAYS_EMPLOYED',
 'DAYS_BIRTH',
 'REGION_POPULATION_RELATIVE',
 'NAME_HOUSING_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_EDUCATION_TYPE',
 'NAME_INCOME_TYPE']

In [11]:
len(items_not_in_list2)

31

In [12]:
pd.set_option('display.max_columns', None)
selected_df = df[items_not_in_list2]
selected_df

,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_CREDIT,AMT_INCOME_TOTAL,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,TARGET,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,HOUR_APPR_PROCESS_START,WEEKDAY_APPR_PROCESS_START,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,ORGANIZATION_TYPE,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE
0,0.593984,360000.0,13059.0,2.0,-3219.0,0,Cash loans,F,360000.0,90000.0,0,0,0,0,0,0,0,8,THURSDAY,2,2,XNA,-4703,-14020.0,365243,-21950,0.014520,House / apartment,Married,Secondary / secondary special,Pensioner
1,0.067658,675000.0,57685.5,2.0,-1244.0,0,Cash loans,F,728460.0,135000.0,0,0,0,0,0,0,0,8,TUESDAY,2,2,School,-1980,-6928.0,-8615,-15645,0.015221,House / apartment,Married,Secondary / secondary special,State servant
2,0.037153,180000.0,9000.0,1.0,-265.0,0,Revolving loans,M,180000.0,225000.0,0,0,0,0,0,0,0,10,MONDAY,2,2,Self-employed,-389,-6740.0,-1049,-11926,0.025164,With parents,Single / not married,Secondary / secondary special,Working
3,0.355276,675000.0,21775.5,2.0,-135.0,1,Cash loans,F,675000.0,72000.0,0,0,0,0,0,0,0,13,THURSDAY,2,2,Self-employed,-2976,-4146.0,-795,-15597,0.019101,House / apartment,Separated,Secondary / secondary special,Working
4,0.656941,450000.0,25407.0,1.0,-1062.0,0,Cash loans,M,545040.0,157500.0,0,0,0,0,0,0,0,12,MONDAY,2,2,Business Entity Type 2,-2010,-2010.0,-1769,-13220,0.006629,House / apartment,Single / not married,Secondary / secondary special,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.421587,679500.0,30280.5,2.0,-709.0,0,Cash loans,F,760225.5,135000.0,0,0,0,0,0,0,0,10,THURSDAY,2,2,School,-4319,-12111.0,-361,-20794,0.022625,House / apartment,Married,Secondary / secondary special,Working
307507,0.352721,270000.0,17383.5,1.0,-2015.0,0,Cash loans,M,270000.0,135000.0,0,0,0,0,0,0,0,12,MONDAY,2,2,Business Entity Type 3,-3883,-5457.0,-672,-11714,0.022625,House / apartment,Single / not married,Secondary / secondary special,Working
307508,0.772387,562500.0,20808.0,2.0,-1543.0,0,Cash loans,M,651600.0,270000.0,0,0,0,0,0,0,0,10,MONDAY,2,2,Self-employed,-5193,-1923.0,-1405,-16424,0.008019,House / apartment,Married,Secondary / secondary special,Working
307509,0.730652,1354500.0,55296.0,2.0,-3146.0,0,Cash loans,M,1354500.0,225000.0,0,0,0,0,0,0,0,11,MONDAY,2,2,Legal Services,-5004,-2380.0,-4587,-14164,0.019689,House / apartment,Married,Higher education,Working


In [13]:
# Extract the 'TARGET' column
target = selected_df['TARGET']

# Remove the 'TARGET' column from the dataframe and reassign the result back to selected_df
selected_df = selected_df.drop(columns=['TARGET'])

# Insert the 'TARGET' column at the beginning of the dataframe
selected_df.insert(0, 'TARGET', target)

### Fix problematic values

Exploring summary statistics of predictors highlighted certain values that did not make sense. In the 'DAYS_EMPLOYED'column, the value of 365243 is repeated frequently and thus causing an innaccurate skewing of the data. We replaced this value with 0 to prevent model inaccuracies.

In [14]:
len(column_list_3)
print(sum(selected_df['DAYS_EMPLOYED']))

19623828581


In [15]:
selected_df.loc[:, 'DAYS_EMPLOYED'] = selected_df['DAYS_EMPLOYED'].replace(365243, 0)
selected_df = selected_df[selected_df['AMT_INCOME_TOTAL'] <= 9000000]

In [16]:
selected_df = selected_df.drop(columns = ['HOUR_APPR_PROCESS_START', 'DAYS_ID_PUBLISH'])
selected_df

,TARGET,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_CREDIT,AMT_INCOME_TOTAL,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,WEEKDAY_APPR_PROCESS_START,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,ORGANIZATION_TYPE,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE
0,0,0.593984,360000.0,13059.0,2.0,-3219.0,0,Cash loans,F,360000.0,90000.0,0,0,0,0,0,0,THURSDAY,2,2,XNA,-14020.0,0,-21950,0.014520,House / apartment,Married,Secondary / secondary special,Pensioner
1,0,0.067658,675000.0,57685.5,2.0,-1244.0,0,Cash loans,F,728460.0,135000.0,0,0,0,0,0,0,TUESDAY,2,2,School,-6928.0,-8615,-15645,0.015221,House / apartment,Married,Secondary / secondary special,State servant
2,0,0.037153,180000.0,9000.0,1.0,-265.0,0,Revolving loans,M,180000.0,225000.0,0,0,0,0,0,0,MONDAY,2,2,Self-employed,-6740.0,-1049,-11926,0.025164,With parents,Single / not married,Secondary / secondary special,Working
3,0,0.355276,675000.0,21775.5,2.0,-135.0,1,Cash loans,F,675000.0,72000.0,0,0,0,0,0,0,THURSDAY,2,2,Self-employed,-4146.0,-795,-15597,0.019101,House / apartment,Separated,Secondary / secondary special,Working
4,0,0.656941,450000.0,25407.0,1.0,-1062.0,0,Cash loans,M,545040.0,157500.0,0,0,0,0,0,0,MONDAY,2,2,Business Entity Type 2,-2010.0,-1769,-13220,0.006629,House / apartment,Single / not married,Secondary / secondary special,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0.421587,679500.0,30280.5,2.0,-709.0,0,Cash loans,F,760225.5,135000.0,0,0,0,0,0,0,THURSDAY,2,2,School,-12111.0,-361,-20794,0.022625,House / apartment,Married,Secondary / secondary special,Working
307507,0,0.352721,270000.0,17383.5,1.0,-2015.0,0,Cash loans,M,270000.0,135000.0,0,0,0,0,0,0,MONDAY,2,2,Business Entity Type 3,-5457.0,-672,-11714,0.022625,House / apartment,Single / not married,Secondary / secondary special,Working
307508,0,0.772387,562500.0,20808.0,2.0,-1543.0,0,Cash loans,M,651600.0,270000.0,0,0,0,0,0,0,MONDAY,2,2,Self-employed,-1923.0,-1405,-16424,0.008019,House / apartment,Married,Secondary / secondary special,Working
307509,0,0.730652,1354500.0,55296.0,2.0,-3146.0,0,Cash loans,M,1354500.0,225000.0,0,0,0,0,0,0,MONDAY,2,2,Legal Services,-2380.0,-4587,-14164,0.019689,House / apartment,Married,Higher education,Working


In [17]:
# Abs value of negatives
selected_df['DAYS_LAST_PHONE_CHANGE'] = selected_df.loc[:, 'DAYS_LAST_PHONE_CHANGE'].abs()
#selected_df['DAYS_ID_PUBLISH'] = selected_df.loc[:, 'DAYS_ID_PUBLISH'].abs()
selected_df['DAYS_REGISTRATION'] = selected_df.loc[:, 'DAYS_REGISTRATION'].abs()
selected_df['DAYS_EMPLOYED'] = selected_df.loc[:, 'DAYS_EMPLOYED'].abs()
selected_df['DAYS_BIRTH'] = selected_df.loc[:, 'DAYS_BIRTH'].abs()

## Preparing Logistic Regression

To prepare for running the logistic regression model, we first had to perform further data manipulation to have it in the proper format. Tasks included imputing all numeric columns, creating an interation term, standerdizing values, and dummy encoding categorical variables.

In [18]:
# Grab numeric columns
numeric_cols = selected_df.select_dtypes(include='number')
numeric_cols

,TARGET,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,AMT_CREDIT,AMT_INCOME_TOTAL,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE
0,0,0.593984,360000.0,13059.0,2.0,3219.0,0,360000.0,90000.0,0,0,0,0,0,0,2,2,14020.0,0,21950,0.014520
1,0,0.067658,675000.0,57685.5,2.0,1244.0,0,728460.0,135000.0,0,0,0,0,0,0,2,2,6928.0,8615,15645,0.015221
2,0,0.037153,180000.0,9000.0,1.0,265.0,0,180000.0,225000.0,0,0,0,0,0,0,2,2,6740.0,1049,11926,0.025164
3,0,0.355276,675000.0,21775.5,2.0,135.0,1,675000.0,72000.0,0,0,0,0,0,0,2,2,4146.0,795,15597,0.019101
4,0,0.656941,450000.0,25407.0,1.0,1062.0,0,545040.0,157500.0,0,0,0,0,0,0,2,2,2010.0,1769,13220,0.006629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0.421587,679500.0,30280.5,2.0,709.0,0,760225.5,135000.0,0,0,0,0,0,0,2,2,12111.0,361,20794,0.022625
307507,0,0.352721,270000.0,17383.5,1.0,2015.0,0,270000.0,135000.0,0,0,0,0,0,0,2,2,5457.0,672,11714,0.022625
307508,0,0.772387,562500.0,20808.0,2.0,1543.0,0,651600.0,270000.0,0,0,0,0,0,0,2,2,1923.0,1405,16424,0.008019
307509,0,0.730652,1354500.0,55296.0,2.0,3146.0,0,1354500.0,225000.0,0,0,0,0,0,0,2,2,2380.0,4587,14164,0.019689


In [19]:
# Create a list of numeric column names excluding target for imputations

column_names = numeric_cols.columns.tolist()
my_list = [x for x in column_names if x != 'TARGET']
print(my_list)

['EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE', 'CNT_CHILDREN', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'LIVE_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_LIVE_REGION', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_POPULATION_RELATIVE']


### N/a Imputer

In [20]:
# Create a SimpleImputer instance
imputer = SimpleImputer(strategy='median')

# Fit and transform the selected numeric columns
selected_df.loc[:, my_list] = imputer.fit_transform(selected_df.loc[:, my_list])

In [21]:
selected_df.isnull().sum()

TARGET                         0
EXT_SOURCE_2                   0
AMT_GOODS_PRICE                0
AMT_ANNUITY                    0
CNT_FAM_MEMBERS                0
DAYS_LAST_PHONE_CHANGE         0
CNT_CHILDREN                   0
NAME_CONTRACT_TYPE             0
CODE_GENDER                    0
AMT_CREDIT                     0
AMT_INCOME_TOTAL               0
LIVE_CITY_NOT_WORK_CITY        0
REG_CITY_NOT_WORK_CITY         0
REG_CITY_NOT_LIVE_CITY         0
LIVE_REGION_NOT_WORK_REGION    0
REG_REGION_NOT_WORK_REGION     0
REG_REGION_NOT_LIVE_REGION     0
WEEKDAY_APPR_PROCESS_START     0
REGION_RATING_CLIENT_W_CITY    0
REGION_RATING_CLIENT           0
ORGANIZATION_TYPE              0
DAYS_REGISTRATION              0
DAYS_EMPLOYED                  0
DAYS_BIRTH                     0
REGION_POPULATION_RELATIVE     0
NAME_HOUSING_TYPE              0
NAME_FAMILY_STATUS             0
NAME_EDUCATION_TYPE            0
NAME_INCOME_TYPE               0
dtype: int64

In [22]:
selected_df

,TARGET,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_CREDIT,AMT_INCOME_TOTAL,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,WEEKDAY_APPR_PROCESS_START,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,ORGANIZATION_TYPE,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE
0,0,0.593984,360000.0,13059.0,2.0,3219.0,0,Cash loans,F,360000.0,90000.0,0,0,0,0,0,0,THURSDAY,2,2,XNA,14020.0,0,21950,0.014520,House / apartment,Married,Secondary / secondary special,Pensioner
1,0,0.067658,675000.0,57685.5,2.0,1244.0,0,Cash loans,F,728460.0,135000.0,0,0,0,0,0,0,TUESDAY,2,2,School,6928.0,8615,15645,0.015221,House / apartment,Married,Secondary / secondary special,State servant
2,0,0.037153,180000.0,9000.0,1.0,265.0,0,Revolving loans,M,180000.0,225000.0,0,0,0,0,0,0,MONDAY,2,2,Self-employed,6740.0,1049,11926,0.025164,With parents,Single / not married,Secondary / secondary special,Working
3,0,0.355276,675000.0,21775.5,2.0,135.0,1,Cash loans,F,675000.0,72000.0,0,0,0,0,0,0,THURSDAY,2,2,Self-employed,4146.0,795,15597,0.019101,House / apartment,Separated,Secondary / secondary special,Working
4,0,0.656941,450000.0,25407.0,1.0,1062.0,0,Cash loans,M,545040.0,157500.0,0,0,0,0,0,0,MONDAY,2,2,Business Entity Type 2,2010.0,1769,13220,0.006629,House / apartment,Single / not married,Secondary / secondary special,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0.421587,679500.0,30280.5,2.0,709.0,0,Cash loans,F,760225.5,135000.0,0,0,0,0,0,0,THURSDAY,2,2,School,12111.0,361,20794,0.022625,House / apartment,Married,Secondary / secondary special,Working
307507,0,0.352721,270000.0,17383.5,1.0,2015.0,0,Cash loans,M,270000.0,135000.0,0,0,0,0,0,0,MONDAY,2,2,Business Entity Type 3,5457.0,672,11714,0.022625,House / apartment,Single / not married,Secondary / secondary special,Working
307508,0,0.772387,562500.0,20808.0,2.0,1543.0,0,Cash loans,M,651600.0,270000.0,0,0,0,0,0,0,MONDAY,2,2,Self-employed,1923.0,1405,16424,0.008019,House / apartment,Married,Secondary / secondary special,Working
307509,0,0.730652,1354500.0,55296.0,2.0,3146.0,0,Cash loans,M,1354500.0,225000.0,0,0,0,0,0,0,MONDAY,2,2,Legal Services,2380.0,4587,14164,0.019689,House / apartment,Married,Higher education,Working


### Interaction Term

In [23]:
selected_df['CREDIT_TO_INCOME'] = selected_df['AMT_CREDIT']/selected_df['AMT_INCOME_TOTAL']

### Standard Scaler to scale dataset

In [24]:
my_list

['EXT_SOURCE_2',
 'AMT_GOODS_PRICE',
 'AMT_ANNUITY',
 'CNT_FAM_MEMBERS',
 'DAYS_LAST_PHONE_CHANGE',
 'CNT_CHILDREN',
 'AMT_CREDIT',
 'AMT_INCOME_TOTAL',
 'LIVE_CITY_NOT_WORK_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'REG_CITY_NOT_LIVE_CITY',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'REG_REGION_NOT_LIVE_REGION',
 'REGION_RATING_CLIENT_W_CITY',
 'REGION_RATING_CLIENT',
 'DAYS_REGISTRATION',
 'DAYS_EMPLOYED',
 'DAYS_BIRTH',
 'REGION_POPULATION_RELATIVE']

In [25]:
scaler = StandardScaler()

# Fit and transform the selected columns
selected_df[my_list] = scaler.fit_transform(selected_df[my_list])

### Dummy Encoder 

In [26]:
# dummy encoding data set

selected_df = pd.get_dummies(selected_df, drop_first=True)

### Splitting Target and Predictors

In [27]:
X = selected_df.drop(columns=['TARGET'])
y = selected_df['TARGET']

## Logistic Regression - Upsampled

Upsampling has a significant impact on the AUC score compared to that of the original logistic regression model, present in our group notebook. The upsampled AUC score is roughly .934, while the AUC of the original regression model is .703. This shows that upsampling the data leads to signifiant improvement of the data set given it's highly imbalanced nature.

In [28]:
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Upsample the dataset
X_upsampled, y_upsampled = smote.fit_resample(X, y)

# Initialize and train the logistic regression model with the upsampled dataset
log_reg_upsampled = LogisticRegression(max_iter=2000)
log_reg_upsampled.fit(X_upsampled, y_upsampled)

# Make predictions on the upsampled dataset
y_pred_upsampled = log_reg_upsampled.predict(X_upsampled)

# Calculate accuracy on the upsampled dataset
accuracy_upsampled = accuracy_score(y_upsampled, y_pred_upsampled)
print("Train Accuracy (Upsampled):", accuracy_upsampled)
print("Train R2 (Upsampled):", log_reg_upsampled.score(X_upsampled, y_upsampled))

Train Accuracy (Upsampled): 0.8835059642569087
Train R2 (Upsampled): 0.8835059642569087


In [29]:
# Predict probabilities for the positive class (class 1) using upsampled model
y_prob_upsampled = log_reg_upsampled.predict_proba(X_upsampled)[:, 1]

# Calculate false positive rate, true positive rate, and thresholds for upsampled model
fpr_upsampled, tpr_upsampled, thresholds_upsampled = roc_curve(y_upsampled, y_prob_upsampled)

# Calculate AUC score for upsampled model
auc_score_upsampled = auc(fpr_upsampled, tpr_upsampled)
print("AUC Score (Upsampled):", auc_score_upsampled)

AUC Score (Upsampled): 0.9399333091931292


### Mean Squared Error - upsampled logistic model

In [ ]:
# calculating mean squared error - upsampled
log_scores_upsampled = cross_val_score(log_reg_upsampled, 
                                       X_upsampled, 
                                       y_upsampled,
                                       scoring="neg_mean_squared_error",
                                       cv=10 # create 10 folds
)

log_rmse_scores_upsampled = np.sqrt(-log_scores_upsampled)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [32]:
# displaying mean squared error
display_scores(log_rmse_scores_upsampled)

Scores: [0.65504339 0.29978384 0.30443828 0.29791951 0.29672973 0.28647888
 0.28155909 0.27849565 0.28184413 0.2869441 ]
Mean: 0.32692366023201835
Standard deviation: 0.10969373634886853


### R squared - upsampled logistic model

In [99]:
log_up_r2_scores = cross_val_score(log_reg_upsampled, 
                             X_upsampled, 
                             y_upsampled,
                             scoring="r2", 
                             cv=10)
display_scores(log_up_r2_scores)

Scores: [-1.00003538 -0.99996463 -0.99996463 -1.00003538 -1.00003538 -1.00003538
 -0.99996463 -1.00003538 -1.         -1.        ]
Mean: -1.000007075513384
Standard deviation: 3.0839294641349914e-05


### AUC - upsampled logistic model

In [82]:
log_up_auc_scores = cross_val_score(log_reg_upsampled, 
                             X_upsampled, 
                             y_upsampled,
                             scoring="roc_auc", 
                             cv=10                               
)
log_up_mean_auc = np.mean(log_up_auc_scores)
display_scores(log_up_auc_scores)

Scores: [0.39590006 0.37154273 0.26070667 0.10165558 0.47117858 0.50665821
 0.39784145 0.36870688 0.30433986 0.25336769]
Mean: 0.3431897719134749
Standard deviation: 0.11182254153778533


### AUC score - upsampled logistic model

## Conclusion

Preliminary model building in my individual notebook highlights the importance of upsampling while working with very imbalanced data sets. Given that over 91% of the data set falls under the target variable 0 (repayment on time), the models do not have much data to train on when it comes to data that represented the minority target varible 1 (or those that did not repay on time).

Feature engineering is an important process when working with models as it can help optimize overall performance. Upsampling showed great success, something that is reflected in our group notebook as well.

This project also provided insight into how vague and sometimes messy data sets can be. Many of the columns had NAs that represented over 50% of the data. Many of the columns also did not provide valuable insight into what they actually represented (nor were the column descriptions very insightful). It required an inquisitive mindset to continue to question and work with the data as well as the models during the building process.